# Data of temperatures in Chile

Author: Hugo Carrillo

Creation: July 2024

This notebook shows the very beginning steps in heatwave detection from meteorological stations data. 
1. Detection of missing data: quantification and cleaning.
2. Selection of stations. 
3. We generate cleaned data for heatwave detections.

## Libraries import

In [2]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) # Suppress specific RuntimeWarnings

In [3]:
%load_ext autoreload
%autoreload 2

# Parameters


In [4]:
METADATA_PATH_AND_FILENAME = 'data/metadata.parquet'
TEMP_DATA_PATH = 'data/original/'
CLEANED_DATA_PATH = 'data/cleaned/'

# Utilities


In [5]:
# this function takes the hourly temperatures and defines the max, min, and mean temperatures.
def to_format(data):
    #data is a pd dataframe containing the hourly temperatures for a given station
    data['date'] = pd.to_datetime(data['date'], format="%Y-%m-%d")
    data.index = data['date']#.dt.date

    max_temp = data.resample('D').max()
    #max_temp = data.resample('D').apply(lambda x: np.nanmax(x))
    max_temp.rename(columns={'temperature': 'max_temp'}, inplace=True)
    max_temp = max_temp.drop(columns=['date'])

    max_temp.index = pd.to_datetime(max_temp.index.date)


    min_temp = data.resample('D').min()
    min_temp.rename(columns={'temperature': 'min_temp'}, inplace=True)
    min_temp = min_temp.drop(columns=['date'])
    min_temp.index = pd.to_datetime(min_temp.index.date)


    mean_temp = data.resample('D').mean()
    mean_temp.rename(columns={'temperature': 'mean_temp'}, inplace=True)
    mean_temp = mean_temp.drop(columns=['date'])
    mean_temp.index = pd.to_datetime(mean_temp.index.date)

    df_station = pd.concat([max_temp, min_temp, mean_temp], axis=1)

    #df_station =  df_station.dropna()

    return df_station

# Reading data

In [6]:
# get all meteochile stations information
stations = pd.read_parquet(METADATA_PATH_AND_FILENAME)
stations.head()

,WMO_code,ICAO_code,name,latitude,longitude,altitude,state_id,geographic_zone_id,data_link
national_code,,,,,,,,,
170001,85403,,Visviri Tenencia,-17.59500,-69.47750,4084,15,0,https://climatologia.meteochile.gob.cl/applica...
180005,85406,SCAR,"Chacalluta, Arica Ap.",-18.35555,-70.34028,50,15,1,https://climatologia.meteochile.gob.cl/applica...
180017,85405,,Putre,-18.20000,-69.56250,3532,15,2,https://climatologia.meteochile.gob.cl/applica...
180018,85407,,"Defensa Civil, Arica",-18.49111,-70.30139,71,15,1,https://climatologia.meteochile.gob.cl/applica...
180042,85408,,"Cerro Sombrero, Arica",-18.51250,-70.26556,122,15,1,https://climatologia.meteochile.gob.cl/applica...


In [7]:
# Reading raw data of selected stations
statlist = [330007, 330020, 330030, 340031, 360011, 380013, 360019, 390006, 400009, 410005]
station_data_to_read = {}
stations_data = {}
for stat in statlist:
    station_data_to_read[stat] = pd.read_parquet(TEMP_DATA_PATH + f'{stat}.parquet')
    station_data_to_read[stat] = station_data_to_read[stat][station_data_to_read[stat]['date'].dt.year > 1970]
    stations_data[stat] = to_format(station_data_to_read[stat])
    #save
    stations_data[stat].dropna().to_parquet(CLEANED_DATA_PATH + f'{stat}.parquet')



In [8]:
# checking the data
print('Temp data:')
print(station_data_to_read[stat].head())
print('-------')
print('-------')
print('Temp data in format:')
print(stations_data[stat].head())



Temp data:
                                               date  temperature
date                                                            
1971-01-01 00:00:00-03:00 1971-01-01 00:00:00-03:00         12.6
1971-01-01 03:00:00-03:00 1971-01-01 03:00:00-03:00          9.2
1971-01-01 06:00:00-03:00 1971-01-01 06:00:00-03:00          8.8
1971-01-01 07:00:00-03:00 1971-01-01 07:00:00-03:00          8.4
1971-01-01 08:00:00-03:00 1971-01-01 08:00:00-03:00         10.8
-------
-------
Temp data in format:
            max_temp  min_temp  mean_temp
1971-01-01      19.2       8.4  15.158824
1971-01-02      15.4      11.6  13.794444
1971-01-03      14.5       8.8  12.100000
1971-01-04      14.0       9.8  11.933333
1971-01-05      16.8      10.3  13.426316


In [9]:
# This cell provides information about the missing and possible cleaning of the data
# In the case of Chile, we found missing data in form of nans. See the table in the output cell.

df_nans = pd.DataFrame(index=statlist)
total_days = np.zeros((len(statlist),))
missing_days = np.zeros((len(statlist),))
missing_hours = np.zeros((len(statlist),))
total_hours = np.zeros((len(statlist),))
missing_daytime = np.zeros((len(statlist),))


for i, stat in enumerate(statlist):
    total_days[i] = len(station_data_to_read[stat].groupby(station_data_to_read[stat].index.date))

    days_with_nan = stations_data[stat].isna().groupby(stations_data[stat].index.date).any()
    nan_days = days_with_nan[days_with_nan['max_temp']].index

    missing_days[i] = len(nan_days)

    other_missings = stations_data[stat][stations_data[stat]['max_temp'] - stations_data[stat]['min_temp'] < 1e-5]
    missing_daytime[i] = len(other_missings)

    days_with_nan = station_data_to_read[stat].isna().groupby(station_data_to_read[stat].index.date).any()
    nan_days = days_with_nan[days_with_nan['temperature']].index
    df_with_nan_days = station_data_to_read[stat][np.isnan(station_data_to_read[stat]['temperature'])]
    missing_hours[i] = len(df_with_nan_days)
    total_hours[i] = len(station_data_to_read[stat])

    days_with_nan = stations_data[stat].isna().groupby(stations_data[stat].index.date).any()
    nan_days = days_with_nan[days_with_nan['max_temp']].index

df_nans['tot. days'] = total_days
df_nans['miss. days'] = missing_days
df_nans['p. miss. days'] = missing_days/total_days
df_nans['miss. hours'] = missing_hours
df_nans['p. miss. hours'] = missing_hours/total_hours

print(df_nans)

        tot. days  miss. days  p. miss. days  miss. hours  p. miss. hours
330007    19532.0        17.0       0.000870       3765.0        0.014001
330020    19577.0         2.0       0.000102         13.0        0.000073
330030    19533.0       405.0       0.020734      29689.0        0.100816
340031    19575.0         8.0       0.000409        701.0        0.002187
360011    19472.0       107.0       0.005495        588.0        0.002035
380013    19411.0       168.0       0.008655        414.0        0.000933
360019    19579.0         0.0       0.000000         56.0        0.000120
390006    19574.0        11.0       0.000562       2390.0        0.008248
400009    19559.0        20.0       0.001023       1618.0        0.005894
410005    19577.0         4.0       0.000204         76.0        0.000163


In [10]:
print(f"Percentage of missing entries={sum(df_nans['miss. hours'])/sum(total_hours)}")
print(f"Percentage of missing days={sum(df_nans['miss. days'])/sum(df_nans['tot. days'])}")

Percentage of missing entries=0.011936402748986044
Percentage of missing days=0.0037975525746075777
